In [38]:
# import numpy as np
import pickle

class Agent():
    
    def __init__(self,move_value):
        self.move_value = move_value
    
    def play(self):
        pass
    
    def update(self):
        pass

class Environment():
    
    def __init__(self):
        self.board = np.zeros((3,3))
    
    # Def: Check if there is any winner
    def check_winner(self):
        for row in self.board:
            if sum(row) == 3: return 1
            if sum(row) ==-3: return -1 
            
        for row in np.transpose(self.board):
            if sum(row) == 3: return 1
            if sum(row) ==-3: return -1 
            
        if sum([self.board[i][i] for i in range(len(self.board))]) == 3:  return 1
        if sum([self.board[i][i] for i in range(len(self.board))]) == -3: return -1
        if sum([self.board[i][len(self.board)-i-1] for i in range(len(self.board))]) == 3: return  1       
        if sum([self.board[i][len(self.board)-i-1] for i in range(len(self.board))]) == -3:return -1        
            
    # Def: Check if there is a winner
    def check_if_there_is_winner(self): return self.check_winner()
    
    # change the borad position - location and value.      
    def make_a_move(self,position,value):
        assert (value == 1 or value == -1)
        self.board.ravel()[position] = value
    
    # Reset the board back the starting position.     
    def reset_game(self): self.board = np.zeros((3,3))
        
    # View the current state of the board
    def get_board(self): return self.board

In [44]:
temp = Environment()
temp.make_a_move(0,1)
temp.make_a_move(1,1)
temp.make_a_move(8,-1)
print(temp.get_board())
print(temp.check_if_there_is_winner())
temp.make_a_move(2,1)
print(temp.get_board())
print(temp.check_if_there_is_winner())


[[ 1.  1.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -1.]]
None
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0. -1.]]
1


In [24]:
import numpy as np

def checkRows(board):
    print(board)
    for row in board:
        if len(set(row)) == 1:
            return row[0]
    return 0

def checkDiagonals(board):
    if len(set([board[i][i] for i in range(len(board))])) == 1:
        return board[0][0]
    if len(set([board[i][len(board)-i-1] for i in range(len(board))])) == 1:
        return board[0][len(board)-1]
    return 0

def checkWin(board):
    #transposition to check rows, then columns
    for newBoard in [board, np.transpose(board)]:
        result = checkRows(newBoard)
        if result:
            return result
    return checkDiagonals(board)


a = [[1, 1, 1],
     ['A', 'A', 'A'],
     ['A', 'X', 'A']]

print(checkWin(a))

[[1, 1, 1], ['A', 'A', 'A'], ['A', 'X', 'A']]
1


In [27]:

def checkRows(board):
    for row in board:
        
        if sum(row) == 3 : return 1
        if sum(row) == -3: return -1
        print(row)
        print(sum(row))
        
        
        
        if len(set(row)) == 1:
            return row[0]
    return 0
checkRows(a)

[1, 1, 1]
3


1

In [45]:
! git all-go

[master a245f88] commit
 2 files changed, 326 insertions(+), 30 deletions(-)
Counting objects: 4, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 2.09 KiB | 2.09 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/JaeDukSeo/tictactoe.git
   a4f7264..a245f88  master -> master
